# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 15 2023 10:40AM,257430,19,9093812,Eywa Pharma Inc.,Released
1,Feb 15 2023 10:36AM,257446,10,PRF-53150,Bio-PRF,Released
2,Feb 15 2023 10:36AM,257446,10,PRF-53153,Bio-PRF,Released
3,Feb 15 2023 10:36AM,257446,10,PRF-53160,Bio-PRF,Released
4,Feb 15 2023 10:36AM,257446,10,PRF-53033,Bio-PRF,Released
5,Feb 15 2023 10:36AM,257446,10,PRF-53036,Bio-PRF,Released
6,Feb 15 2023 10:36AM,257446,10,PRF-53039,Bio-PRF,Released
7,Feb 15 2023 10:36AM,257446,10,PRF-53042,Bio-PRF,Released
8,Feb 15 2023 10:36AM,257446,10,PRF-53046,Bio-PRF,Released
9,Feb 15 2023 10:36AM,257446,10,PRF-53147,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
54,257441,Released,29
55,257442,Released,38
56,257444,Released,32
57,257445,Released,2
58,257446,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257441,NaN,NaN,29.0
257442,NaN,NaN,38.0
257444,NaN,NaN,32.0
257445,NaN,NaN,2.0
257446,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257339,0.0,1.0,0.0
257340,34.0,9.0,1.0
257341,0.0,0.0,2.0
257342,0.0,3.0,2.0
257346,0.0,3.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257339,0,1,0
257340,34,9,1
257341,0,0,2
257342,0,3,2
257346,0,3,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257339,0,1,0
1,257340,34,9,1
2,257341,0,0,2
3,257342,0,3,2
4,257346,0,3,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257339,,1,
1,257340,34,9,1
2,257341,,,2
3,257342,,3,2
4,257346,,3,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.
1,Feb 15 2023 10:36AM,257446,10,Bio-PRF
12,Feb 15 2023 10:35AM,257442,10,ISDIN Corporation
50,Feb 15 2023 10:34AM,257444,10,ISDIN Corporation
82,Feb 15 2023 10:34AM,257441,10,ISDIN Corporation
111,Feb 15 2023 10:34AM,257440,10,ISDIN Corporation
139,Feb 15 2023 10:32AM,257445,12,Hush Hush
141,Feb 15 2023 10:30AM,257439,10,ISDIN Corporation
144,Feb 15 2023 10:21AM,257438,15,"Virtus Pharmaceuticals, LLC"
145,Feb 15 2023 10:20AM,257437,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,,,1
1,Feb 15 2023 10:36AM,257446,10,Bio-PRF,,,11
2,Feb 15 2023 10:35AM,257442,10,ISDIN Corporation,,,38
3,Feb 15 2023 10:34AM,257444,10,ISDIN Corporation,,,32
4,Feb 15 2023 10:34AM,257441,10,ISDIN Corporation,,,29
5,Feb 15 2023 10:34AM,257440,10,ISDIN Corporation,,,28
6,Feb 15 2023 10:32AM,257445,12,Hush Hush,,,2
7,Feb 15 2023 10:30AM,257439,10,ISDIN Corporation,,,3
8,Feb 15 2023 10:21AM,257438,15,"Virtus Pharmaceuticals, LLC",,,1
9,Feb 15 2023 10:20AM,257437,15,"Person & Covey, Inc.",,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,1,,
1,Feb 15 2023 10:36AM,257446,10,Bio-PRF,11,,
2,Feb 15 2023 10:35AM,257442,10,ISDIN Corporation,38,,
3,Feb 15 2023 10:34AM,257444,10,ISDIN Corporation,32,,
4,Feb 15 2023 10:34AM,257441,10,ISDIN Corporation,29,,
5,Feb 15 2023 10:34AM,257440,10,ISDIN Corporation,28,,
6,Feb 15 2023 10:32AM,257445,12,Hush Hush,2,,
7,Feb 15 2023 10:30AM,257439,10,ISDIN Corporation,3,,
8,Feb 15 2023 10:21AM,257438,15,"Virtus Pharmaceuticals, LLC",1,,
9,Feb 15 2023 10:20AM,257437,15,"Person & Covey, Inc.",4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,1,,
1,Feb 15 2023 10:36AM,257446,10,Bio-PRF,11,,
2,Feb 15 2023 10:35AM,257442,10,ISDIN Corporation,38,,
3,Feb 15 2023 10:34AM,257444,10,ISDIN Corporation,32,,
4,Feb 15 2023 10:34AM,257441,10,ISDIN Corporation,29,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,1.0,NaN,NaN
1,Feb 15 2023 10:36AM,257446,10,Bio-PRF,11.0,NaN,NaN
2,Feb 15 2023 10:35AM,257442,10,ISDIN Corporation,38.0,NaN,NaN
3,Feb 15 2023 10:34AM,257444,10,ISDIN Corporation,32.0,NaN,NaN
4,Feb 15 2023 10:34AM,257441,10,ISDIN Corporation,29.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,1.0,0.0,0.0
1,Feb 15 2023 10:36AM,257446,10,Bio-PRF,11.0,0.0,0.0
2,Feb 15 2023 10:35AM,257442,10,ISDIN Corporation,38.0,0.0,0.0
3,Feb 15 2023 10:34AM,257444,10,ISDIN Corporation,32.0,0.0,0.0
4,Feb 15 2023 10:34AM,257441,10,ISDIN Corporation,29.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,8236663,254.0,45.0,35.0
12,772299,3.0,1.0,0.0
15,1287052,51.0,6.0,1.0
19,1544355,4.0,3.0,0.0
21,514865,2.0,0.0,0.0
22,772178,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,8236663,254.0,45.0,35.0
1,12,772299,3.0,1.0,0.0
2,15,1287052,51.0,6.0,1.0
3,19,1544355,4.0,3.0,0.0
4,21,514865,2.0,0.0,0.0
5,22,772178,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,254.0,45.0,35.0
1,12,3.0,1.0,0.0
2,15,51.0,6.0,1.0
3,19,4.0,3.0,0.0
4,21,2.0,0.0,0.0
5,22,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,254.0
1,12,Released,3.0
2,15,Released,51.0
3,19,Released,4.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Completed,35.0,0.0,1.0,0.0,0.0,0.0
Executing,45.0,1.0,6.0,3.0,0.0,1.0
Released,254.0,3.0,51.0,4.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Completed,35.0,0.0,1.0,0.0,0.0,0.0
1,Executing,45.0,1.0,6.0,3.0,0.0,1.0
2,Released,254.0,3.0,51.0,4.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Completed,35.0,0.0,1.0,0.0,0.0,0.0
1,Executing,45.0,1.0,6.0,3.0,0.0,1.0
2,Released,254.0,3.0,51.0,4.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()